In [1]:
import os
from dotenv import load_dotenv, find_dotenv
import openai
def get_openai_key():
    _ = load_dotenv(find_dotenv())
    return os.environ['OPENAI_API_KEY']

openai.api_key = get_openai_key()
def get_completion(prompt, model="gpt-4"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

# 提示原则

## 一、 编写清晰、具体的指令

### 1.1 使用分割符清晰的表示输入的不同部分,可以选择```，"""，< >，<tag> </tag>，: 等做分隔符，只要能明确起到隔断作用即可

In [3]:
text = f"""
您应该提供尽可能清晰、具体的指示，以表达您希望模型执行的任务。\
这将引导模型朝向所需的输出，并降低收到无关或不正确响应的可能性。\
不要将写清晰的提示词与写简短的提示词混淆。\
在许多情况下，更长的提示词可以为模型提供更多的清晰度和上下文信息，从而导致更详细和相关的输出。
"""
# 需要总结的文本内容
prompt = f"""
把用三个反引号括起来的文本总结成一句话。
```{text}```
"""
response=get_completion(prompt)
response

'这段文本强调了提供清晰、具体指示的重要性，以引导模型生成所需的输出，同时指出长的提示词可以提供更多的清晰度和上下文信息，从而得到更详细和相关的输出。'

### 1.2结构化输出

In [4]:
prompt = f"""
请生成包括书名、作者和类别的三本虚构的、非真实存在的中文书籍清单，\
并以 JSON 格式提供，其中包含以下键:book_id、title、author、genre。
"""
response = get_completion(prompt)
response


[
  {
    "book_id": "001",
    "title": "月亮下的秘密",
    "author": "李华",
    "genre": "神秘惊悚"
  },
  {
    "book_id": "002",
    "title": "时间的尽头",
    "author": "张三",
    "genre": "科幻小说"
  },
  {
    "book_id": "003",
    "title": "爱情的味道",
    "author": "王小二",
    "genre": "浪漫小说"
  }
]


### 1.3要求模型检查是否满足条件

In [5]:
# 满足条件的输入（text中提供了步骤）
text_1 = f"""
泡一杯茶很容易。首先，需要把水烧开。\
在等待期间，拿一个杯子并把茶包放进去。\
一旦水足够热，就把它倒在茶包上。\
等待一会儿，让茶叶浸泡。几分钟后，取出茶包。\
如果您愿意，可以加一些糖或牛奶调味。\
就这样，您可以享受一杯美味的茶了。
"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Text 1 的总结:")
response


Text 1 的总结:


'第一步 - 把水烧开。\n第二步 - 在等待期间，拿一个杯子并把茶包放进去。\n第三步 - 一旦水足够热，就把它倒在茶包上。\n第四步 - 等待一会儿，让茶叶浸泡。\n第五步 - 几分钟后，取出茶包。\n第六步 - 如果您愿意，可以加一些糖或牛奶调味。\n第七步 - 就这样，您可以享受一杯美味的茶了。'

### 1.4提供少量示例


In [6]:
prompt = f"""
您的任务是以一致的风格回答问题。

<孩子>: 请教我何为耐心。

<祖父母>: 挖出最深峡谷的河流源于一处不起眼的泉眼；最宏伟的交响乐从单一的音符开始；最复杂的挂毯以一根孤独的线开始编织。

<孩子>: 请教我何为韧性。
"""
response = get_completion(prompt)
response


'<祖父母>: 那棵在风暴中屹立不倒的橡树，它的根深深地扎入土壤；那只在寒冷冬季中生存的北极熊，它的毅力和坚韧让它能够在极端环境中生存；那个在困难中不断尝试的人，他的韧性让他能够克服一切困难。'

## 二、给模型时间思考

### 2.1指定完成任务所需的步骤

In [8]:
prompt_2 = f"""
1-用一句话概括下面用<>括起来的文本。
2-将摘要翻译成英语。
3-在英语摘要中列出每个名称。
4-输出一个 JSON 对象，其中包含以下键：English_summary，num_names。

请使用以下格式：
文本：<要总结的文本>
摘要：<摘要>
翻译：<摘要的翻译>
名称：<英语摘要中的名称列表>
输出 JSON：<带有 English_summary 和 num_names 的 JSON>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nprompt 2:")
print(response)



prompt 2:
摘要：<兄妹杰克和吉尔在去山顶井打水的过程中遭遇了意外，但他们依然保持了冒险精神，继续愉快地探索。>
翻译：<Siblings Jack and Jill encountered an accident while fetching water from a well on a mountain top, but they still maintained their adventurous spirit and continued to explore happily.>
名称：<Jack, Jill>
输出 JSON：<{"English_summary": "Siblings Jack and Jill encountered an accident while fetching water from a well on a mountain top, but they still maintained their adventurous spirit and continued to explore happily.", "num_names": 2}>


### 2.2指导模型在下结论之前找出一个自己的解法

In [9]:
prompt = f"""
请判断学生的解决方案是否正确，请通过如下步骤解决这个问题：

步骤：

    首先，自己解决问题。
    然后将您的解决方案与学生的解决方案进行比较，对比计算得到的总费用与学生计算的总费用是否一致，并评估学生的解决方案是否正确。
    在自己完成问题之前，请勿决定学生的解决方案是否正确。

使用以下格式：

    问题：问题文本
    学生的解决方案：学生的解决方案文本
    实际解决方案和步骤：实际解决方案和步骤文本
    学生计算的总费用：学生计算得到的总费用
    实际计算的总费用：实际计算出的总费用
    学生计算的费用和实际计算的费用是否相同：是或否
    学生的解决方案和实际解决方案是否相同：是或否
    学生的成绩：正确或不正确

问题：

    我正在建造一个太阳能发电站，需要帮助计算财务。 
    - 土地费用为每平方英尺100美元
    - 我可以以每平方英尺250美元的价格购买太阳能电池板
    - 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元;

    作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：

    设x为发电站的大小，单位为平方英尺。
    费用：
    1. 土地费用：100x美元
    2. 太阳能电池板费用：250x美元
    3. 维护费用：100,000+100x=10万美元+10x美元
    总费用：100x美元+250x美元+10万美元+100x美元=450x+10万美元

实际解决方案和步骤：
"""
response = get_completion(prompt)
response


'设x为发电站的大小，单位为平方英尺。\n费用：\n1. 土地费用：100x美元\n2. 太阳能电池板费用：250x美元\n3. 维护费用：100,000+10x=10万美元+10x美元\n总费用：100x美元+250x美元+10万美元+10x美元=360x+10万美元\n\n学生计算的总费用：450x+10万美元\n实际计算的总费用：360x+10万美元\n学生计算的费用和实际计算的费用是否相同：否\n学生的解决方案和实际解决方案是否相同：否\n学生的成绩：不正确'

## 三、 局限性

### 3.1虚假知识：模型偶尔会生成一些看似真实实则编造的知识